In [ ]:
import requests
import re

class UserContextServer(MCPBaseServer):
    def __init__(self):
        super().__init__("user-context", "user_profile")

    def authenticate(self, request):
        # Verify OAuth 2.1 token with user_profile scope
        token = request.headers.get("Authorization").split(" ")[1]
        jwt.decode(
            token,
            options={"verify_aud": False, "verify_iss": True},
            issuer="https://accounts.google.com",
            audience="mcp-user-service",
            scopes=["user_profile"]
        )

    def fetch_data(self, request):
        # Get API key from Secret Manager
        api_key = self.secret_client.access_secret_version(
            name="projects/my-project/secrets/API_KEY/versions/latest"
        ).payload.data.decode()

        # Fetch from REST API (GenAI-generated from OpenAPI spec)
        user_id = request.json["user_id"]
        if not re.match(r"^[a-f\d]{8}-([a-f\d]{4}-){3}[a-f\d]{12}$", user_id):
            raise ValueError("Invalid user ID format")

        response = requests.get(
            f"https://api.userservice.com/v1/profiles/{user_id}",
            headers={"Authorization": f"Bearer {api_key}"}
        )
        return response.json()

    def build_context(self, raw_data):
        # Convert to JSON-LD for agent reasoning
        return {
            "@context": "https://schema.org/",
            "@type": "Person",
            "name": f"{raw_data['firstName']} {raw_data['lastName']}",
            "email": raw_data['contact']['email'],
            "metadata": {
                "source": "REST_API",
                "freshness": datetime.datetime.utcnow().isoformat(),
                "size_kb": len(json.dumps(raw_data)) / 1024
            }
        }